## Import Requirements and Datasets

In [1]:
# requirements
import numpy as np
import pandas as pd

In [2]:
# dataset 1
df_data = pd.read_csv('original_datasets/data.csv')
df_data.head()

,Date,Selina,Sub Category,Capacity,Out of Order,Available Beds,Sold Beds,Room Revenue USD,Food and Beverage Revenue,Activities Revenue,Co-Working Revenue,Average Guest Review
0,2017-12-01,San Jose,Dorm,86,1,85,65,1170.0,341.640,179.010,79.560,8.755
1,2017-12-01,Medellin,Dorm,215,25,190,91,1820.0,373.100,274.820,72.800,8.787
2,2017-12-01,Jaco,Dorm,118,2,116,55,880.0,289.520,100.320,23.760,8.780
3,2017-12-01,Bocas del Toro,Dorm,154,2,152,108,1728.0,618.624,390.528,164.160,8.689
4,2017-12-01,Red Frog,Dorm,24,1,23,17,255.0,91.545,44.880,20.655,8.754


In [3]:
# dataset 2
df_op_dates = pd.read_csv('original_datasets/opening_dates.csv')
df_op_dates.head()

,Selina,Selina Country,Selina Opening Date
0,Venao,Panama,2014-08-01
1,Bocas del Toro,Panama,2014-12-01
2,Pedasi,Panama,2016-03-01
3,San Jose,Costa Rica,2016-11-01
4,Red Frog,Panama,2016-11-01


## Assessing

### Visual Assessment

#### List of Visual Assessments:

- Contains Selinas that opened outside of Q4 2017-2018 date range (ref. opening dates):
    - Are all Selina's listed considered "newly opened" and we're simply analyzing their 2018 performance?
    - Or do we remove all Selinas that opened prior to the data collection period?
- Missing columns:
    - Total revenue
    - Selina's KPIs
- Some Selina's are missing chunks of data for certain bed types and some are missing individual rows of data for certain bed types:
    - Does this mean Selina didn't have this bed type available?
    - Or maybe this bed type wasn't booked for that day?
- Column names not in snake_case
- Missing categorical columns that could potentially add value
    - Country
    - Month of year each Selina opened

### Programmatic Assessment

In [4]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19077 entries, 0 to 19076
Data columns (total 12 columns):
Date                         19077 non-null object
Selina                       19077 non-null object
Sub Category                 19077 non-null object
Capacity                     19077 non-null int64
Out of Order                 19077 non-null int64
Available Beds               19077 non-null int64
Sold Beds                    19077 non-null int64
Room Revenue USD             19077 non-null float64
Food and Beverage Revenue    19077 non-null float64
Activities Revenue           19077 non-null float64
Co-Working Revenue           19077 non-null float64
Average Guest Review         19077 non-null float64
dtypes: float64(5), int64(4), object(3)
memory usage: 1.7+ MB


In [5]:
df_data.duplicated().sum()

0

In [6]:
df_data.describe()

,Capacity,Out of Order,Available Beds,Sold Beds,Room Revenue USD,Food and Beverage Revenue,Activities Revenue,Co-Working Revenue,Average Guest Review
count,19077.000000,19077.000000,19077.000000,19077.000000,19077.000000,19077.000000,19077.000000,19077.000000,19077.000000
mean,64.614195,4.362426,60.251769,31.359228,629.480657,151.040265,92.938102,32.068546,8.778343
std,53.954401,5.809417,49.804572,25.606965,411.518606,134.996453,74.167449,29.188877,0.091046
min,6.000000,0.000000,5.000000,2.000000,36.000000,0.000000,0.000000,0.000000,8.685000
25%,24.000000,1.000000,23.000000,13.000000,324.000000,57.214350,42.882840,13.547520,8.717000
50%,40.000000,1.000000,39.000000,22.000000,540.000000,120.669120,72.421020,23.332608,8.753000
75%,96.000000,5.000000,86.000000,41.000000,820.800000,207.820800,122.068800,41.164200,8.785000
max,254.000000,43.000000,233.000000,126.000000,2541.000000,900.191250,541.479920,277.397500,9.010000


In [7]:
# check if all df_data Selinas are in df_op_dates DataFrame, or if any Selinas need to be removed from df_data
print(len(df_data.Selina.unique()))

count = 0

for selina in df_data.Selina.unique():
    if selina in df_op_dates.Selina.unique():
        count += 1
    else:
        count += 0

print(count)

23
23


#### List of Programmatic Assessments:
- No NaNs
    - If we add rows for the missing data recognized in the visual assessment, we will create NaNs that need to be filled accordingly
- Date column wrong dtype
- No duplicates

## Cleaning

### List of Cleaning Issues:

#### df_data:
1. Convert column names to snake_case.
2. Convert date column to datetime dtype.
3. Create new columns for total_revenue and the missing KPIs of occupancy_rate and trevpob.
4. Create new categorical columns for country and month_of_opening (month of the year that the Selina opened).
5. Create new rows for the missing data for each room type at each Selina by separating into separate DataFrames by Selina.
6. Fill in NaNs from newly added rows in each separate DataFrame.
7. Merge the separate DataFrames back into a master dataset now that all rows have been added and NaNs filled.

#### df_op_dates:
1. 
2. 

### Issue 1:

#### Define:
Convert column names to snake_case using .replace() in order to make Python functions and methods more efficient.

#### Code:

In [8]:
# both DataFrames
dfs = [df_data, df_op_dates]

for df in dfs:
    df.columns = df.columns.str.lower().str.replace(" ","_").str.replace("-","_")

#### Test:

In [9]:
df_data.columns

Index(['date', 'selina', 'sub_category', 'capacity', 'out_of_order',
       'available_beds', 'sold_beds', 'room_revenue_usd',
       'food_and_beverage_revenue', 'activities_revenue', 'co_working_revenue',
       'average_guest_review'],
      dtype='object')

In [10]:
df_op_dates.columns

Index(['selina', 'selina_country', 'selina_opening_date'], dtype='object')

### Issue 2:

#### Define:
Convert date column to datetime dtype.

#### Code:

In [11]:
# both dfs
df_data['date'] = pd.to_datetime(df_data['date'])
df_op_dates['selina_opening_date'] = pd.to_datetime(df_op_dates['selina_opening_date'])

#### Test:

In [12]:
df_data.date.dtypes

dtype('<M8[ns]')

In [13]:
df_op_dates.selina_opening_date.dtypes

dtype('<M8[ns]')

### Issue 3:

#### Define:
Create new columns for total_revenue, occupancy_rate, and trevpob.

#### Code:

In [14]:
# total_revenue
df_data['total_revenue'] = df_data.apply(lambda x: x.room_revenue_usd + x.food_and_beverage_revenue + x.activities_revenue + x.co_working_revenue, axis=1)

# trevpob
df_data['trevpob'] = df_data.total_revenue / df_data.sold_beds

# occupancy rate
df_data['occupancy_rate'] = df_data.sold_beds / df_data.available_beds

In [15]:
# re-order the columns to their appropriate index
df_data = df_data[['date', 'selina', 'sub_category', 'capacity', 'out_of_order', 'available_beds',
                   'sold_beds', 'occupancy_rate', 'room_revenue_usd','food_and_beverage_revenue',
                   'activities_revenue', 'co_working_revenue', 'total_revenue', 'trevpob','average_guest_review']]

#### Test:

In [16]:
df_data.head()

,date,selina,sub_category,capacity,out_of_order,available_beds,sold_beds,occupancy_rate,room_revenue_usd,food_and_beverage_revenue,activities_revenue,co_working_revenue,total_revenue,trevpob,average_guest_review
0,2017-12-01,San Jose,Dorm,86,1,85,65,0.764706,1170.0,341.640,179.010,79.560,1770.210,27.234,8.755
1,2017-12-01,Medellin,Dorm,215,25,190,91,0.478947,1820.0,373.100,274.820,72.800,2540.720,27.920,8.787
2,2017-12-01,Jaco,Dorm,118,2,116,55,0.474138,880.0,289.520,100.320,23.760,1293.600,23.520,8.780
3,2017-12-01,Bocas del Toro,Dorm,154,2,152,108,0.710526,1728.0,618.624,390.528,164.160,2901.312,26.864,8.689
4,2017-12-01,Red Frog,Dorm,24,1,23,17,0.739130,255.0,91.545,44.880,20.655,412.080,24.240,8.754


### Save Progress to .csv File

In [17]:
df_data.to_csv('clean_datasets/data-clean-v1.csv', index=False)
df_op_dates.to_csv('clean_datasets/opening-dates-clean-v1.csv', index=False)

### Issue 4:

#### Define:
Left join the categorical columns from df_op_dates. Join on the selina name and bring over both columns from df_op_dates, country will remain as is and opening date will be converted into a month of year column.

#### Code:

In [18]:
df_data = df_data.merge(df_op_dates, on='selina', how='left')

In [19]:
df_data = df_data[['date', 'selina', 'selina_country', 'selina_opening_date', 'sub_category',
                   'capacity', 'out_of_order', 'available_beds', 'sold_beds', 'occupancy_rate',
                   'room_revenue_usd', 'food_and_beverage_revenue', 'activities_revenue',
                   'co_working_revenue', 'total_revenue', 'trevpob', 'average_guest_review']]

#### Test:

In [20]:
df_data.head()

,date,selina,selina_country,selina_opening_date,sub_category,capacity,out_of_order,available_beds,sold_beds,occupancy_rate,room_revenue_usd,food_and_beverage_revenue,activities_revenue,co_working_revenue,total_revenue,trevpob,average_guest_review
0,2017-12-01,San Jose,Costa Rica,2016-11-01,Dorm,86,1,85,65,0.764706,1170.0,341.640,179.010,79.560,1770.210,27.234,8.755
1,2017-12-01,Medellin,Colombia,2017-06-01,Dorm,215,25,190,91,0.478947,1820.0,373.100,274.820,72.800,2540.720,27.920,8.787
2,2017-12-01,Jaco,Costa Rica,2017-06-01,Dorm,118,2,116,55,0.474138,880.0,289.520,100.320,23.760,1293.600,23.520,8.780
3,2017-12-01,Bocas del Toro,Panama,2014-12-01,Dorm,154,2,152,108,0.710526,1728.0,618.624,390.528,164.160,2901.312,26.864,8.689
4,2017-12-01,Red Frog,Panama,2016-11-01,Dorm,24,1,23,17,0.739130,255.0,91.545,44.880,20.655,412.080,24.240,8.754


#### Code:

In [21]:
# create month_of_opening and year_of_opening columns using opening_date column
df_data['month_of_opening'] = df_data['selina_opening_date'].dt.month

In [22]:
# re-order columns
df_data = df_data[['date', 'selina', 'selina_country', 'selina_opening_date', 'month_of_opening',
                   'sub_category', 'capacity', 'out_of_order', 'available_beds', 'sold_beds',
                   'occupancy_rate', 'room_revenue_usd', 'food_and_beverage_revenue', 'activities_revenue',
                   'co_working_revenue', 'total_revenue', 'trevpob', 'average_guest_review']]

#### Test:

In [23]:
df_data.head()

,date,selina,selina_country,selina_opening_date,month_of_opening,sub_category,capacity,out_of_order,available_beds,sold_beds,occupancy_rate,room_revenue_usd,food_and_beverage_revenue,activities_revenue,co_working_revenue,total_revenue,trevpob,average_guest_review
0,2017-12-01,San Jose,Costa Rica,2016-11-01,11,Dorm,86,1,85,65,0.764706,1170.0,341.640,179.010,79.560,1770.210,27.234,8.755
1,2017-12-01,Medellin,Colombia,2017-06-01,6,Dorm,215,25,190,91,0.478947,1820.0,373.100,274.820,72.800,2540.720,27.920,8.787
2,2017-12-01,Jaco,Costa Rica,2017-06-01,6,Dorm,118,2,116,55,0.474138,880.0,289.520,100.320,23.760,1293.600,23.520,8.780
3,2017-12-01,Bocas del Toro,Panama,2014-12-01,12,Dorm,154,2,152,108,0.710526,1728.0,618.624,390.528,164.160,2901.312,26.864,8.689
4,2017-12-01,Red Frog,Panama,2016-11-01,11,Dorm,24,1,23,17,0.739130,255.0,91.545,44.880,20.655,412.080,24.240,8.754


### Issue 5:

#### Define:
Convert categorical columns to the correct data type.

#### Code:

In [24]:
columns_to_convert_to_cat = ['selina', 'selina_country', 'month_of_opening']

for column in columns_to_convert_to_cat:
    df_data[column] = pd.Categorical(df_data[column])

#### Test:

In [25]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19077 entries, 0 to 19076
Data columns (total 18 columns):
date                         19077 non-null datetime64[ns]
selina                       19077 non-null category
selina_country               19077 non-null category
selina_opening_date          19077 non-null datetime64[ns]
month_of_opening             19077 non-null category
sub_category                 19077 non-null object
capacity                     19077 non-null int64
out_of_order                 19077 non-null int64
available_beds               19077 non-null int64
sold_beds                    19077 non-null int64
occupancy_rate               19077 non-null float64
room_revenue_usd             19077 non-null float64
food_and_beverage_revenue    19077 non-null float64
activities_revenue           19077 non-null float64
co_working_revenue           19077 non-null float64
total_revenue                19077 non-null float64
trevpob                      19077 non-null float64

### Save Progress to .csv File

In [26]:
df_data.to_csv('clean_datasets/data-clean-v2.csv', index=False)

### Issue 6:

#### Define:
Create a performance index column in order to combine our 3 KPIs into a single peformance score. Since the number of KPIs is small, the performance score will provide an accurate representation of overall performance.

The performance index will convert each KPI into a percentage score out of 100%
- Occupancy rate is already a score out of 100%
- TREVPOB does not have a maximum possible value (like occupancy rate does), so it will be given a percentage score relative to the values in the dataset. The score will be calculated by dividing the actual value in each row dividied by the maximum value observed in the dataset.
- Average guest review is a rank out of 10, which could easily be converted into a percentage score out of 100%. However, in doing this all of our values will be scores ranging from 86.8% to 90.1%, which is too small a range to accurately represent the important distinction between a 9.0 rating (i.e. a 90% rating) and 8.0 rating when customers are booking our rooms. Therefore, this will converted into a new column classifying reviews as greater than or less than 9.0.

#### Code:

In [27]:
# create review score over 9.0 column
df_data['review_over_9'] = df_data['average_guest_review'] >= 9

#### Test:

In [28]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19077 entries, 0 to 19076
Data columns (total 19 columns):
date                         19077 non-null datetime64[ns]
selina                       19077 non-null category
selina_country               19077 non-null category
selina_opening_date          19077 non-null datetime64[ns]
month_of_opening             19077 non-null category
sub_category                 19077 non-null object
capacity                     19077 non-null int64
out_of_order                 19077 non-null int64
available_beds               19077 non-null int64
sold_beds                    19077 non-null int64
occupancy_rate               19077 non-null float64
room_revenue_usd             19077 non-null float64
food_and_beverage_revenue    19077 non-null float64
activities_revenue           19077 non-null float64
co_working_revenue           19077 non-null float64
total_revenue                19077 non-null float64
trevpob                      19077 non-null float64

#### Code:

In [29]:
# create necessary calculations for performance index
occupancy_rate_perfect_score = 1
trevpob_perfect_score = df_data.trevpob.max()

#### Test:

In [30]:
print(occupancy_rate_perfect_score, trevpob_perfect_score)

1 67.65473000000001


#### Code:

In [31]:
# calculate performance index
aggregate_score = (df_data.occupancy_rate / occupancy_rate_perfect_score) + (df_data.trevpob / trevpob_perfect_score) + (df_data.review_over_9)

performance_index = aggregate_score/3

df_data['performance_index'] = performance_index

#### Test:

In [32]:
df_data.head()

,date,selina,selina_country,selina_opening_date,month_of_opening,sub_category,capacity,out_of_order,available_beds,sold_beds,occupancy_rate,room_revenue_usd,food_and_beverage_revenue,activities_revenue,co_working_revenue,total_revenue,trevpob,average_guest_review,review_over_9,performance_index
0,2017-12-01,San Jose,Costa Rica,2016-11-01,11,Dorm,86,1,85,65,0.764706,1170.0,341.640,179.010,79.560,1770.210,27.234,8.755,False,0.389083
1,2017-12-01,Medellin,Colombia,2017-06-01,6,Dorm,215,25,190,91,0.478947,1820.0,373.100,274.820,72.800,2540.720,27.920,8.787,False,0.297210
2,2017-12-01,Jaco,Costa Rica,2017-06-01,6,Dorm,118,2,116,55,0.474138,880.0,289.520,100.320,23.760,1293.600,23.520,8.780,False,0.273928
3,2017-12-01,Bocas del Toro,Panama,2014-12-01,12,Dorm,154,2,152,108,0.710526,1728.0,618.624,390.528,164.160,2901.312,26.864,8.689,False,0.369200
4,2017-12-01,Red Frog,Panama,2016-11-01,11,Dorm,24,1,23,17,0.739130,255.0,91.545,44.880,20.655,412.080,24.240,8.754,False,0.365807
